In [ ]:
# !python3 -m spacy download en_core_web_lg

In [35]:
import joblib, numpy as np

In [36]:
from pprint import pprint

In [37]:
import spacy

In [38]:
import torch

In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [40]:
if torch.cuda.is_available():
   device = torch.device("cuda:1")
   print("Running on the GPU")
else:
   device = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [41]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [52]:
def get_question(answer, context, max_length=64):
    input_text = "answer: %s  context: %s </s>" % (answer, context)
    features = tokenizer([input_text], return_tensors='pt')
    output = model.generate(input_ids=features['input_ids'], 
                            attention_mask=features['attention_mask'],
                            num_beams=50,
                            no_repeat_ngram_size=2,
                            num_return_sequences=1,
                            early_stopping=True,
                            max_length=max_length)
    questions = [tokenizer.decode(o) for o in output]
    return questions

context = "This picture describes about inside view of a hall, in this we can find group of people,\
few are sitting on the chairs, in the background we can see lights."
answer = "not many"


get_question(answer, context)

# output: question: Who created the RuPERTa-base

['<pad> question: How many people are in the hall?</s>']

In [78]:
context = 'In this image there are cushions on the bed. \
Below it there are baskets. Background there is a wall having windows. \
From the windows few trees are visible. Left side there is a blue curtain.'
answer="blue"

get_question(answer, context)


['<pad> question: What color is the curtain on the left side of the bed?</s>']

In [133]:
c = 'there is a shelf'
a = 'yes'

get_question(a, c)

['<pad> question: Is there a shelf?</s>']

In [88]:
context = "This picture describes about inside view of a hall, in this we can find group of people,\
few are sitting on the chairs, in the background we can see lights."
answer = "chairs"

get_question(answer, context)

['<pad> question: What do some people sit on in the picture?</s>']

In [140]:
context = "In the background I can see few curtains, a door, few candle stands, a fireplace, two lamps"
answer = "curtains"

get_question(answer, context)

['<pad> question: What do I see in the background?</s>']

In [102]:
context =  'In this picture I can see 3 stools and the countertops on the floor in front.\
On a countertop, I can see few things. \
In the background, I can see the wall and a cupboard. \
On the top of this picture, I can see the lights on the ceiling. \
On the left side of this picture, I ca can see a white color thing.'
answer = "left"

get_question(answer, context)

['<pad> question: On which side of the picture can I see a white thing?</s>']

In [46]:
import spacy
activated = spacy.prefer_gpu()

nlp = spacy.load("en_core_web_lg")

In [93]:
## answers

doc = nlp(context)
pos_answers = []

print(doc,"\n")
for word in doc:
    if word.tag_ in ['NN','JJ','NNS'] \
    and word.text.lower() not in ['side','background','picture','image']:
        pos_answers.append(word.text)
        print(word.text, word.tag_, word.dep_, ", Explanation:", spacy.explain(word.tag_))

In this picture I can see 3 stools and the countertops on the floor in front. On a countertop, I can see few things. In the background, I can see the wall and a cupboard. On the top of this picture, I can see the lights on the ceiling. On the left side of this picture, I ca can see a white color thing. 

stools NNS dobj , Explanation: noun, plural
countertops NNS conj , Explanation: noun, plural
floor NN pobj , Explanation: noun, singular or mass
front NN pobj , Explanation: noun, singular or mass
countertop NN pobj , Explanation: noun, singular or mass
few JJ amod , Explanation: adjective (English), other noun-modifier (Chinese)
things NNS dobj , Explanation: noun, plural
wall NN dobj , Explanation: noun, singular or mass
cupboard NN conj , Explanation: noun, singular or mass
top NN pobj , Explanation: noun, singular or mass
lights NNS dobj , Explanation: noun, plural
ceiling NN pobj , Explanation: noun, singular or mass
left JJ amod , Explanation: adjective (English), other noun-modi

In [103]:
# dependency_answers = []
# print(doc,"\n")
# for word in doc:
#     if word.tag_ in ['NN','JJ','NNS']: 
#         if word.dep_ in ['nsubj','obj','pobj','dobj']:
#             dependency_answers.append(word.text)
#             print(word.text, word.dep_, ", Explanation:", spacy.explain(word.dep_))


In [94]:
pos_questions = []
for i in pos_answers:
    pos_questions.append(get_question(i, context))
for i, j in zip(pos_questions, pos_answers):
    pprint(i)
    pprint(j)

['<pad> question: What do I see in the picture?</s>']
'stools'
['<pad> question: What is on the floor in front of the stools?</s>']
'countertops'
['<pad> question: Where are the countertops in front of the stools?</s>']
'floor'
['<pad> question: In what part of the house are the countertops on the '
 'floor?</s>']
'front'
['<pad> question: On what surface can I see few things?</s>']
'countertop'
['<pad> question: How many things can I see on a countertop?</s>']
'few'
['<pad> question: What can I see on a countertop?</s>']
'things'
['<pad> question: Along with a cupboard, what is in the background of the '
 'picture?</s>']
'wall'
['<pad> question: What is in the background of the picture?</s>']
'cupboard'
['<pad> question: On what side of the picture are the lights on the '
 'ceiling?</s>']
'top'
['<pad> question: On the top of the picture, what is on the ceiling?</s>']
'lights'
['<pad> question: On the top of the picture, where are the lights?</s>']
'ceiling'
['<pad> question: On which

In [ ]:
bad_answers = ['side','top','color','thing','background','foreground',
               
              ]
bad_questions = ['what do i see ..',
                ]
maybe_bad = ['few','many','some'
            ]

In [49]:
cand_ans = []
for chunk in doc.noun_chunks:
#     print(chunk.text)
    cand_ans.append(chunk.text)
cand_ans = set(cand_ans)
print(cand_ans)

noun_chunk_questions = []
for i in cand_ans:
    noun_chunk_questions.append(get_question(i, context))

{'windows', 'baskets', 'a wall', 'a curtain', 'this image', 'cushions', 'the bed', 'it', 'side', 'Background', 'few trees', 'the windows'}


In [50]:
from pprint import pprint
for i, j in zip(cand_ans, noun_chunk_questions):
    pprint(i)
    pprint(j)

'windows'
['<pad> question: What is on the wall below the bed?</s>',
 '<pad> question: What is on the wall above the bed?</s>',
 '<pad> question: What is on the wall of the bed?</s>']
'baskets'
['<pad> question: What is below the bed in the image?</s>',
 '<pad> question: What is below the bed?</s><pad><pad><pad>',
 '<pad> question: What is below the bed in this image?</s>']
'a wall'
['<pad> question: What has windows on it?</s><pad><pad><pad>',
 '<pad> question: What is the background of the bed?</s><pad>',
 '<pad> question: What is on the background of the bed?</s>']
'a curtain'
['<pad> question: What is on the left side of the bed?</s><pad>',
 '<pad> question: What is visible on the left side of the bed?</s>',
 '<pad> question: On the left side of the bed is what?</s><pad>']
'this image'
['<pad> question: In what image are cushions on the bed?</s><pad>',
 '<pad> question: In what image are cushions on a bed?</s>',
 '<pad> question: In which image are cushions on the bed?</s><pad>']
'

In [7]:
ade20k = joblib.load("../data/ADE20K-pairs/imgs/index_ade20k.pkl")

In [9]:
ade20k.keys()

dict_keys(['filename', 'folder', 'objectIsPart', 'objectPresence', 'objectcounts', 'objectnames', 'proportionClassIsPart', 'scene', 'wordnet_found', 'wordnet_level1', 'wordnet_synset', 'wordnet_hypernym', 'wordnet_gloss', 'wordnet_frequency', 'description'])

In [33]:
ade20k['description']

{'filename': 'array of length N=27574 with the image file names',
 'folder': 'array of length N with the image folder names.',
 'scene': 'array of length N providing the scene name (same classes as the Places database) for each image.',
 'objectIsPart': 'array of size [C, N] counting how many times an object is a part in each image. objectIsPart[c,i]=m if in image i object class c is a part of another object m times. For objects, objectIsPart[c,i]=0, and for parts we will find: objectIsPart[c,i] = objectPresence(c,i)',
 'objectPresence': 'array of size [C, N] with the object counts per image. objectPresence(c,i)=n if in image i there are n instances of object class c.',
 'objectcounts': 'array of length C with the number of instances for each object class.',
 'objectnames': 'array of length C with the object class names.',
 'proportionClassIsPart': 'array of length C with the proportion of times that class c behaves as a part. If proportionClassIsPart[c]=0 then it means that this is a 

In [30]:
pprint(len(ade20k['objectnames']))
print(ade20k['objectnames'])

3688
['-', 'aarm panel', 'abacus', 'accordion, piano accordion, squeeze box', 'acropolis', 'ad, advertisement, advertizement, advertising, advertizing, advert', 'adding machine', 'advertisement board', 'aerial', 'air conditioner, air conditioning', 'air hockey table', 'air machine', 'aircraft carrier', 'airplane, aeroplane, plane', 'airport cart', 'alarm', 'alarm clock', 'alembic', 'alga', 'algae', "altar, communion table, Lord's table", 'altarpiece', 'amphitheater', 'amphora', 'anchor', 'andiron', 'andirons', 'animal toy', 'animal, animate being, beast, brute, creature, fauna', 'animals', 'antenna', 'antenna, aerial, transmitting aerial', 'antler', 'antlers', 'anvil', 'aperture', 'apparatus', 'apparel, wearing apparel, dress, clothes', 'apple', 'apples', 'appliance', 'apron', 'aquarium', 'aqueduct', 'arbor', 'arcade', 'arcade machine', 'arcade machines', 'arcade, colonnade', 'arcades', 'arch', 'arch, archway', 'arches', 'arm', 'arm panel', 'arm support', 'armchair', 'armor', 'armrest'

In [130]:
ade20k['objectnames'][255]

'bouquet, corsage, posy, nosegay'

In [129]:
[i for i, object in enumerate(ade20k['objectPresence'][255]) if object!=0]

[2648,
 2783,
 2820,
 2850,
 2855,
 2867,
 2875,
 2882,
 2921,
 2925,
 2933,
 2959,
 2981,
 3470,
 3736,
 3759,
 3859,
 3864,
 3967,
 3975,
 3999,
 4001,
 4009,
 4040,
 4065,
 4074,
 4087,
 4097,
 4114,
 4161,
 5990,
 6018,
 6024,
 6040,
 6052,
 6867,
 6886,
 6888,
 6926,
 6981,
 6993,
 7006,
 7023,
 7024,
 7029,
 7039,
 7057,
 7064,
 7070,
 7073,
 7080,
 7081,
 7105,
 7113,
 7125,
 7136,
 7147,
 7148,
 7150,
 10204,
 10253,
 10298,
 10309,
 10313,
 10349,
 10381,
 10432,
 10447,
 10491,
 10521,
 10522,
 10596,
 10620,
 10665,
 10668,
 11064,
 11145,
 11147,
 11150,
 11163,
 11172,
 11196,
 11206,
 11207,
 13316,
 13539,
 13626,
 14057,
 14059,
 14440]

In [26]:
pprint(len(ade20k['objectIsPart']))
print(ade20k['objectIsPart'])

3688
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [29]:
len(ade20k['objectIsPart'][0])

27574

In [31]:
pprint(len(ade20k['objectPresence']))
print(ade20k['objectPresence'])

3688
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [32]:
len(ade20k['objectPresence'][0])

27574